In [1]:
import pandas as pd
import os
import numpy as np
import datetime
from datetime import timedelta
import fnmatch

In [2]:
## As what greg did###
#1. Load an allied health file
#2. Ensure allied health file as examdate
#3. Do a right type join (assume left = og file, right = allied health file)
#4. Extract rows only with a T1 file path, so we have full rows
#5. Calculate abs(allied health data - MRI visit date) and label it like df.datediffs or some shit
#6. df.groupby with PTID, and then sort by datediffs ascend
#7. Keep the row where datadiff is less than 28 days and the smallest val. If tie, then we ball and just choose at random
#8. Conduct a final length of rows check - MUST BE 15733
#9. Rinse and repeat

In [3]:
df = pd.read_csv('/rds/general/project/c3nl_scott_students/ephemeral/sankeith/gregbased_scott10k_alliedhealth.csv', low_memory = False)
df[f'VISDATE'] = pd.to_datetime(df[f'VISDATE'])
df[f'EXAMDATE_4WKS_LATER'] = pd.to_datetime(df[f'EXAMDATE_4WKS_LATER'])
df[f'EXAMDATE_4WKS_B4'] = pd.to_datetime(df[f'EXAMDATE_4WKS_B4'])
print(len(df))
print(len(df.columns))
print(len(df['PTID'].unique()))

15733
14
2635


In [21]:
vars = [
    ['/rds/general/project/c3nl_scott_students/live/sankeith/scott_10k_housekeeping/scott_10k_extra_adni_info/DXSUM_10Sep2025.csv',['PTID','EXAMDATE','DIAGNOSIS'], {'DIAGNOSIS':np.nan}],
    ['/rds/general/project/c3nl_scott_students/live/sankeith/scott_10k_housekeeping/scott_10k_extra_adni_info/MMSE_28Mar2025.csv', ['PTID','EXAMDATE','MMSCORE'], {'MMSCORE':np.nan}],
    ['/rds/general/project/c3nl_scott_students/live/sankeith/scott_10k_housekeeping/scott_10k_extra_adni_info/GDSCALE_28Mar2025.csv',['PTID','EXAMDATE''GDTOTAL'], {'GDTOTAL':np.nan}],
    ['/rds/general/project/c3nl_scott_students/live/sankeith/scott_10k_housekeeping/scott_10k_extra_adni_info/MOCA_30Mar2025.csv', ['PTID','EXAMDATE', 'MOCA'], {'MOCA':np.nan}],
    ['/rds/general/project/c3nl_scott_students/live/sankeith/scott_10k_housekeeping/scott_10k_extra_adni_info/NEUROBAT_28Mar2025.csv', ['PTID','EXAMDATE','LIMMTOTAL', 'CLOCKSCOR', 'LDELTOTAL', 'LDELCUE', 'ANART'], {'LIMMTOTAL':np.nan, 'CLOCKSCOR':np.nan, 'LDELTOTAL':np.nan, 'LDELCUE':np.nan, 'ANART':np.nan}]
]
for i in range(0,len(vars)):
    column_suffix = vars[i][0].split('/')[-1]
    column_suffix = column_suffix.split('_')[0]
    varnames = vars[i][1]
    
    #Load an allied health file
    Talli = pd.read_csv(vars[i][0], low_memory = False)
    
    if 'RID' in Talli.columns:
        Talli.drop(columns = 'RID', inplace = True)    
    print(f"Length of {column_suffix}: {len(Talli)}")
    
    #Ensure allied health file as examdate
    if 'VISDATE' in Talli.columns:
        Talli.rename(columns = {'VISDATE':f"EXAMDATE_{column_suffix}"}, inplace = True)
        Talli[f'EXAMDATE_{column_suffix}'] = pd.to_datetime(Talli[f'EXAMDATE_{column_suffix}'], errors = 'coerce')
    else:
        Talli[f'EXAMDATE'] = pd.to_datetime(Talli[f'EXAMDATE'], errors = 'coerce')

    #Do a right type join
    Tsub = pd.merge(df,Talli, on = 'PTID', how = 'left')
    print(f"Length of the freshly joined dataframe: {len(Tsub)}")
    
    #Extract rows only with a T1 file path, so we have full rows
    Tsub.dropna(subset='T1_PATH',inplace=True)
    print(f"After dropping rows without T1 paths: {len(Tsub)}")

    #Calculate abs(allied health data - MRI visit date) and label it like df.datediffs or some shit
    try:
        Tsub[f'DATEDIFFS_{column_suffix}'] = abs(Tsub[f'EXAMDATE_{column_suffix}'] - Tsub['VISDATE'])
    except KeyError:
        Tsub[f'DATEDIFFS_{column_suffix}'] = abs(Tsub['EXAMDATE'] - Tsub['VISDATE'])

    #df.groupby with PTID, and then sort by datediffs ascend. Select the/a row where DATEDIFFS is the smallest
    Tsub = Tsub.sort_values(f'DATEDIFFS_{column_suffix}', ascending=True).groupby('T1_PATH').head(1)
    Tsub.reset_index(inplace = True)
    for index, row in Tsub.iterrows():
        if row[f'DATEDIFFS_{column_suffix}'] > pd.Timedelta(28, "D"):
            row[f'DATEDIFFS_{column_suffix}'] = pd.NaT
            for key in vars[i][2]:
                row[key] = vars[i][2].get(key)
    print(len(Tsub))
    print(type(Tsub))
    #print(Tsub.head())
    print(Tsub[f'DATEDIFFS_{column_suffix}'].describe())



Length of DXSUM: 15674
Length of the freshly joined dataframe: 112212
After dropping rows without T1 paths: 112212
28 days 00:00:00
15733
<class 'pandas.core.frame.DataFrame'>
count                         15630
mean     25 days 17:19:36.046065259
std      54 days 17:50:27.723053279
min                 0 days 00:00:00
25%                 2 days 00:00:00
50%                10 days 00:00:00
75%                27 days 00:00:00
max              1477 days 00:00:00
Name: DATEDIFFS_DXSUM, dtype: object
Length of MMSE: 14119
Length of the freshly joined dataframe: 105119
After dropping rows without T1 paths: 105119
28 days 00:00:00
15733
<class 'pandas.core.frame.DataFrame'>
count                         15688
mean     17 days 10:01:18.939316675
std      36 days 04:59:25.741913746
min                 0 days 00:00:00
25%                 0 days 00:00:00
50%                 5 days 00:00:00
75%                17 days 00:00:00
max              1470 days 00:00:00
Name: DATEDIFFS_MMSE, dtype: object


2635